In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import cProfile #for checking the nr of calls and execution time
import pstats
from pstats import SortKey

from tqdm.notebook import tqdm
import numpy as np
import multiple_planets_gas_acc as code_gas
from functions_pebble_accretion import *
from functions import *
import functions_plotting as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import pandas as pd
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator, MultipleLocator, AutoMinorLocator
from matplotlib import cm, ticker
from matplotlib import colors
import matplotlib.gridspec as gridspec
import matplotlib.patches as patch
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.lines as mlines 
import sim_loader as sim_load



In [ ]:
params = code_gas.Params(H_r_model = 'irradiated')
sim_params = code_gas.SimulationParams()
r = np.geomspace(1e-1, 50, 100)
t = 0.1 
r0 = 0.1
mstar = (np.linspace(0.1, 10, 100)*u.M_sun).to(u.M_earth).value

fig, axs = plt.subplots(1, 2, figsize=(12, 6))
axs[0].loglog(r, M0_pla(r, t, sigma_gas_steady_state(r, t, params), params), color='gold', label = 'initial mass'+str(params.H_r_model))
axs[0].axhline(8.314566896275828e-06, color='black', linestyle='--')
axs[0].loglog(r, M0_pla(r, t, sigma_gas_steady_state(r, t, params), params), color='gold', label = 'initial mass'+str(params.H_r_model))

for m in mstar:

    params = code_gas.Params(H_r_model = 'irradiated', star_mass = m, M_dot_star=0)
    print('star mass', params.star_mass*u.M_earth.to(u.M_sun))
    axs[1].scatter(m*u.M_earth.to(u.M_sun), M0_pla(r0, t, sigma_gas_steady_state(r0, t, params), params), color='gold', label =str(params.star_mass))

for i in range(2):
    axs[i].tick_params(axis='both', which='major', direction = 'in', labelsize=14, size = 10)
    axs[i].tick_params(axis='both', which='minor', direction = 'in', labelsize=14, size = 8)
    axs[i].grid(True, which='both', linestyle='--', alpha=0.5)
    axs[i].set_title('Initial mass of embros', fontsize=16)

axs[0].set_xlabel(r'$r$ [AU]', fontsize=16)
axs[1].set_xlabel(r'$M_{\star} [M_{\odot}]$', fontsize=16)

axs[0].legend()
axs[0].set_ylabel(r'$M_{peb}$ [M$_\oplus$]', fontsize=16)

In [ ]:
t1 = 1
t = 0.1
def mstar_scaling(mstar):
    return ((mstar**(-4/7))**3)*mstar
Mstar = np.geomspace(0.1, 10, 100)*u.M_sun.to(u.M_earth)
fig, axs =plt.subplots(1, 1, figsize=(6, 6))
for i, mstar in enumerate(Mstar):
    params = code_gas.Params(H_r_model = 'irradiated', star_mass = mstar, M_dot_star=0)
    axs.scatter(Mstar[i]*u.M_earth.to(u.M_sun), (M_dot_star_t_Mstar(t, params)*u.M_earth/u.Myr).to(u.M_sun/u.yr), color='gold', label = "t = "+str(t)+" Myr" if i ==0 else None)
    axs.scatter(Mstar[i]*u.M_earth.to(u.M_sun), (M_dot_star_t_Mstar(t1, params)*u.M_earth/u.Myr).to(u.M_sun/u.yr), color='orange', label = "t = "+str(t1)+" Myr" if i ==0 else None)

#axs.loglog(mstar, mstar_scaling(mstar), color='gold')
axs.set_xlabel(r'$M_{\star} [M_{\odot}]$', fontsize=16)
axs.set_yscale('log')
axs.set_xscale('log')
axs.set_ylabel(r'$\dot{M}_{\star} [M_{\odot}/yr]$', fontsize=16)
axs.tick_params(axis='both', which='major', direction = 'in', labelsize=14, size = 10)
axs.tick_params(axis='both', which='minor', direction = 'in', labelsize=14, size = 8)
axs.grid(True, which='both', linestyle='--', alpha=0.5)
axs.legend()
plt.savefig('figures/Mdotstar_scaling_1Myr.png', dpi=300, bbox_inches='tight')


In [ ]:
12-7

In [ ]:
params = code_gas.Params(H_r_model = 'irradiated')
sim_params = code_gas.SimulationParams()
r = np.geomspace(1e-1, 50, 100)
t = 0.2
c = ['gold', 'navy']
ls = ['-', '--']
#fig, axs = plt.subplots(1, 2, figsize=(12, 6))
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
for i, params in enumerate([code_gas.Params(H_r_model = 'irradiated'), code_gas.Params(H_r_model = 'Lambrechts_mixed')]):
    axs[0].loglog(r, M0_pla(r, t, sigma_gas_steady_state(r, t, params), params), color=c[i], label = 'initial mass, '+str(params.H_r_model))
    axs[1].loglog(r, M_3D_2DB_trans(r, t, params), color='navy',  linestyle = ls[i], label = '3D/2D Bondi accretion new, '+str(params.H_r_model))
    axs[1].loglog(r, M_3D_2DH_trans(r, t, params), color='green', linestyle = ls[i],  label = '3D/2D Hill accretion new, '+str(params.H_r_model))
    axs[1].loglog(r, M_3D_2D_Bondi_new(r, t, params), color='gold', linestyle = ls[i],  label = '3D/2D Bondi accretion new, '+str(params.H_r_model))


axs[0].axhline(8.314566896275828e-06, color='black', linestyle='--')
axs[0].tick_params(axis='both', which='major', direction = 'in', labelsize=14, size = 10)
axs[0].tick_params(axis='both', which='minor', direction = 'in', labelsize=14, size = 8)
axs[0].set_xlabel(r'$r$ [AU]', fontsize=16)
axs[0].set_ylabel(r'$M_{peb}$ [M$_\oplus$]', fontsize=16)
axs[0].set_title('Initial mass of embros', fontsize=16)
axs[0].legend()
axs[1].legend()


In [ ]:

sim_irr_1 = sim_load.JSONtoSimRes("sims/gas_acc/accretion_maps/irr_vfrag/simulation_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json")
sim_irr_10 = sim_load.JSONtoSimRes("sims/gas_acc/accretion_maps/irr_vfrag/simulation_irradiated_e_el_0.01_vfrag_10.0_planets_50_t0_0.1_N_steps10000.json")
sim_params_irr_1 = sim_load.load_sim_params('sims/gas_acc/accretion_maps/irr_vfrag/sim_params_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json')
sim_params_irr_10 = sim_load.load_sim_params('sims/gas_acc/accretion_maps/irr_vfrag/sim_params_irradiated_e_el_0.01_vfrag_10.0_planets_50_t0_0.1_N_steps10000.json')
params_irr_1 = sim_load.load_params('sims/gas_acc/accretion_maps/irr_vfrag/params_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json')
params_irr_10 = sim_load.load_params('sims/gas_acc/accretion_maps/irr_vfrag/params_irradiated_e_el_0.01_vfrag_10.0_planets_50_t0_0.1_N_steps10000.json')



params_dict = {'St_const': None, 
                'M_dot_star': None,
                'v_frag':(1 * u.m/u.s).to(u.au/u.Myr).value,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'iceline_alpha_change': False,
                'iceline_flux_change': False,
                }

t_initial = 0.2
a_p0 = np.geomspace(2,2, num = 1)
t0= [t_initial] * np.ones(len(a_p0))# warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
t_in = 0.2
t_fin = 10
sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in,
                't_fin':t_fin
                }
params = code_gas.Params(**params_dict, H_r_model='irradiated', epsilon_el=1, epsilon_heat=1)

def M_3D_2DH_trans_debug_drift_ep(position, t, params):

    stokes_peb = st_drift_epstein(position, t, params)

    return 3/4*(2*H_peb(stokes_peb, position, t, params)*(np.sqrt(2*np.pi))/np.pi)**3*(omega_k(position, params)**2/(G*stokes_peb))

def M_3D_2DH_trans_debug_drift_st(position, t, params):

    stokes_peb = st_drift_stokes(position, t, params)

    return 3/4*(2*H_peb(stokes_peb, position, t, params)*(np.sqrt(2*np.pi))/np.pi)**3*(omega_k(position, params)**2/(G*stokes_peb))
def M_3D_2DH_trans_debug_frag(position, t, params):

    stokes_peb = st_frag(position, t, params)

    return 3/4*(2*H_peb(stokes_peb, position, t, params)*(np.sqrt(2*np.pi))/np.pi)**3*(omega_k(position, params)**2/(G*stokes_peb))


# print(M_3D_2DH_trans(sim.position[:,0].value, sim_params.t_in, params))
fig, axs =plt.subplots(1,1, figsize = (8,8))
axs.loglog(sim_irr_10.position[:,0].to(u.au), M_3D_2DH_trans_debug_drift_ep(sim_irr_10.position[:,0].value, sim_params_irr_10.t_fin, params), label = 'st drift epstein', linestyle = '--')
axs.loglog(sim_irr_10.position[:,0].to(u.au), M_3D_2DH_trans_debug_drift_st(sim_irr_10.position[:,0].value, sim_params_irr_10.t_fin, params), label = 'st drift stokes', linestyle = '--')
axs.loglog(sim_irr_10.position[:,0].to(u.au), M_3D_2DH_trans_debug_frag(sim_irr_10.position[:,0].value, sim_params_irr_10.t_fin, params), label = 'st fragt', linestyle = '--') 
axs.loglog(sim_irr_10.position[:,0].to(u.au), M_3D_2DH_trans(sim_irr_10.position[:,0].value, sim_params_irr_10.t_fin, params), label = 'stokes ', color = 'black')

axs.set_ylim(1e-6, 1e2)
axs.set_xlabel('r [AU]', fontsize = 25, labelpad=20)
axs.set_ylabel('$M_{\mathrm{3D/2D}}$ [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plot.all_y_ticks(axs, 100)
axs.legend()
plt.savefig("figures/transition_mass.png", bbox_inches='tight')

### Pebble isolation mass

In [ ]:
r = np.geomspace(4e-3, 1e2, num=1000)
t = 0.3
params = code_gas.Params(H_r_model = 'irradiated', star_mass=0.1*const.M_sun.cgs.to(u.M_earth).value, M_dot_star=0)

fig, axs = plt.subplots(1,1, figsize = (8,8))
#axs.loglog(r, peb_iso_liu(r, t, params), label = str(params.H_r_model)+", $M_{\star}$= "+str(params.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$, Liu et al. 2019",  color = 'black')
#axs.loglog(r, M_peb_iso_Bitsch(r, t, params),label = str(params.H_r_model)+", $M_{\star}$= "+str(params.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$, Bitsch et al. 2018")

params_list = [code_gas.Params(H_r_model = 'irradiated', star_mass=0.1*const.M_sun.cgs.to(u.M_earth).value, M_dot_star=0), code_gas.Params(H_r_model = 'irradiated', star_mass=0.55*const.M_sun.cgs.to(u.M_earth).value, M_dot_star=0), code_gas.Params(H_r_model = 'irradiated', star_mass=1*const.M_sun.cgs.to(u.M_earth).value, M_dot_star=0)]

for params in params_list:
    axs.loglog(r, M_peb_iso(r, t, params),label = str(params.H_r_model)+", $M_{\star}$= "+str(params.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$")

axs.grid(True, which='both', linestyle='--', alpha=0.5)

axs.set_xlabel("r [AU]", size = 15)
axs.set_ylabel("$M [M_{\oplus}]$", size = 15)
axs.tick_params(axis='both', which = 'major', size = 10, direction='in')
axs.tick_params(axis='both', which = 'minor', size = 8, direction='in')
axs.legend()
axs.set_title("Pebble isolation mass, $M_{\star} = 0.1 M_{\odot}$", size = 15)
axs.legend()
plt.savefig("figures/pebble_iso.png", bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(1,1, figsize = (8,8))
Mstar = (np.geomspace(0.01,1, num =100)*u.M_sun).to(u.M_earth).value
ice =[]
c = mpl.cm.Blues(np.linspace(0.1, 1, 10))
times = np.linspace(0.1,1, num = 10)

for j,t in enumerate(times):
    for i,m in enumerate(Mstar):
        params = code_gas.Params(H_r_model = 'irradiated', star_mass=m, M_dot_star=0)
        #r_ice = iceline_irr(t, 170, params)
        r_ice = 0.1
        print('r_ice', r_ice)
        ice.append(r_ice)
        axs.scatter((Mstar[i]*u.M_earth).to(u.M_sun), M0_pla_Mstar(r_ice, t, sigma_gas_steady_state(r_ice, t, params), params), color = c[j],label = "t = "+str(t)[:3]+" Myr" if i ==0 else None)
        #axs.scatter((Mstar[i]*u.M_earth).to(u.M_sun), M0_pla_Mstar(r_ice, t1, sigma_gas_steady_state(r_ice, t1, params), params), color = 'orange', label = "t = "+str(t1)+" Myr" if i ==0 else None)

plot.all_y_ticks(axs, num_ticks=100)
axs.grid(True, which='both', linestyle='--', alpha=0.5)
axs.set_xlabel("$M_{\star} [M_{\odot}]$", size = 15)
axs.set_ylabel("$M_0 [M_{\oplus}]$", size = 15)
axs.set_xscale('log')
axs.set_yscale('log')
axs.legend()
axs.tick_params(axis='both', which = 'major', size = 10, direction='in')
axs.tick_params(axis='both', which = 'minor', size = 8, direction='in')
axs.set_title("Initial mass, irradiated, $a_{p,0}$ = ["+str(ice[0])[:4]+"," +str(ice[-1])[:4]+"] AU", size = 15)
plt.savefig("figures/initial_mass_mstar_irrad_01au.png", bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(1,1, figsize = (8,8))
position = (np.geomspace(0.1, 50, num =100)*u.au).value
t = 0.1
t1=1
ice =[]
for i,r in enumerate(position):
    params = code_gas.Params(H_r_model = 'irradiated', star_mass=0.1*const.M_sun.cgs.to(u.M_earth).value, M_dot_star=0)
    axs.scatter(position[i], M0_pla_Mstar(r, t, sigma_gas_steady_state(r, t, params), params), color = 'gold',label = "t = "+str(t)+" Myr" if i ==0 else None)
    print('star mass', params.star_mass*u.M_earth.to(u.M_sun))
    axs.scatter(position[i], M0_pla_Mstar(r, t1, sigma_gas_steady_state(r, t1, params), params), color = 'orange', label = "t = "+str(t1)+" Myr" if i ==0 else None)

plot.all_y_ticks(axs, num_ticks=100)
axs.grid(True, which='both', linestyle='--', alpha=0.5)
axs.set_xlabel("r [AU]", size = 15)
axs.set_ylabel("$M_0 [M_{\oplus}]$", size = 15)
axs.set_xscale('log')
axs.set_yscale('log')
axs.legend()
axs.tick_params(axis='both', which = 'major', size = 10, direction='in')
axs.tick_params(axis='both', which = 'minor', size = 8, direction='in')
axs.set_title("Initial mass, irradiated, $M_{\star} = 0.1 M_{\odot}$  ", size = 15)#["+str(ice[0])[:4]+"," +str(ice[-1])[:4]+"] AU", size = 15)
plt.savefig("figures/initial_mass_pos_irrad.png", bbox_inches='tight')